In [14]:
import av
import cv2
import numpy as np

In [15]:
images_av = []
images_cv2 = []


In [16]:
%%timeit
container = av.open("./videos/WUzgd7C1pWA.mp4")
for frame in container.decode(video=0):
    images_av.append(frame.to_rgb().to_ndarray())



455 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

188 ms ± 24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
images_av = []
images_cv2 = []
container = av.open("./videos/WUzgd7C1pWA.mp4")
for frame in container.decode(video=0):
    images_av.append(frame.to_rgb().to_ndarray())

cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

In [19]:

print(len(images_av), len(images_cv2))

327 327


In [ ]:
correct = 0
assert len(images_cv2) == len(images_av)
for i in range(len(images_cv2)):
    correct += np.array_equal(images_av[i], images_cv2[i])

correct/len(images_cv2)